In [1]:
## Tennis WTA Ranking scrapper
## Author: Bartlomiej Kielech
## Date: 2024/04/29

import requests
from tabulate import tabulate
from bs4 import BeautifulSoup
from openpyxl import Workbook

# Download content
url = 'https://www.tennislive.net/wta/ranking/'
response = requests.get(url)
html_content = response.text

# Parse HTML
soup = BeautifulSoup(html_content, 'html.parser')

# Find dropdown element
dropdown = soup.find('select', id='rankD')

# Find all dropdown options
options = dropdown.find_all('option')

# Create new excel spreadsheet
wb = Workbook()
ws = wb.active

# Iterate over options
for option in options:
    # Set the value of dropdown to actual
    dropdown_value = option['value']
    
    url_full = url+'EN/'+dropdown_value
    response_full = requests.get(url_full)
    html_content_full = response_full.text

    soup_full = BeautifulSoup(html_content_full, 'html.parser')

    # Find the table with class "table_pranks"
    table = soup_full.find('table', class_='table_pranks')
    
    # Counter for tracking the number of rows added to Excel
    row_count = 0

    # Loop through rows in the table and insert into Excel
    for row in table.find_all('tr')[1:]:
        # Limit to 10 rows
        if row_count >= 10:
            break

        # Extract data from each cell in the row
        row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]

        # Add current date to each row
        row_data.append(dropdown_value)

        # Insert row into Excel worksheet
        ws.append(row_data)

        # Increment row count
        row_count += 1

    # Save the Excel workbook
    wb.save('table_data.xlsx')